<a href="https://colab.research.google.com/github/paolanustes/thesis/blob/main/Generate_input_dataset_VF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Input dataset pipeline for dams' detection models


# Import external resources

## Install and import main libraries

References https://github.com/giswqs/geemap 

In [ ]:
# !pip -q install qiskit

In [ ]:
!pip -q install -U geemap
!pip -q install geopandas shapely

     |████████████████████████████████| 450kB 13.7MB/s 
     |████████████████████████████████| 102kB 6.8MB/s 
     |████████████████████████████████| 5.1MB 29.5MB/s 
     |████████████████████████████████| 143kB 49.9MB/s 
     |████████████████████████████████| 81kB 5.9MB/s 
     |████████████████████████████████| 225kB 58.3MB/s 
     |████████████████████████████████| 102kB 7.4MB/s 
     |████████████████████████████████| 1.3MB 42.6MB/s 
     |████████████████████████████████| 2.5MB 42.5MB/s 
     |████████████████████████████████| 1.2MB 48.7MB/s 
     |████████████████████████████████| 122kB 41.7MB/s 
     |████████████████████████████████| 71kB 5.7MB/s 
     |████████████████████████████████| 552kB 42.5MB/s 
     |████████████████████████████████| 122kB 52.5MB/s 
     |████████████████████████████████| 378kB 48.0MB/s 
     |████████████████████████████████| 71kB 5.7MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.12.1 which is incompatible.
ER

In [ ]:
import ee
import geemap.eefolium as geemap
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=JtFGUCwp-KzYyfON-mPUGTOrEZOl2At6V8TfqZhGPuE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5TkToxVUxUgCpfA53pXrsjoL3Wvf7pynFv99zNvU712vG5f3Hq6S0

Successfully saved authorization token.


In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive') #To access google drive folders
GDRIVE='/content/drive/MyDrive/Thesis'

Mounted at /content/drive


In [ ]:
from shapely.geometry import Point
from shapely.geometry import CAP_STYLE
from pandas import json_normalize
import requests
import pandas as pd
import geopandas
import csv
import sys

## Add Earth Engine data (Control group: California, USA)

1.   Satellite imagery NAIP https://www.fsa.usda.gov/programs-and-services/aerial-photography/imagery-programs/naip-imagery/
2.   Digital Elevation Model NED https://www.usgs.gov/core-science-systems/national-geospatial-program/national-map
3. Boundaries by state TIGER https://www.census.gov/programs-surveys/geography/guidance/tiger-data-products-guide.html
4. Water occurrence JRC http://global-surface-water.appspot.com/ 
5. Continent's boundaries users/gena/land_polygons_image 






In [ ]:
#Create an interactive map
Map = geemap.Map()

In [ ]:
#input number you want to search
state = input('Enter state postal code: (Example for california enter CA)\n')

#read csv, and split on "," the line
csv_file = csv.reader(open(f'{GDRIVE}/FIPS.csv', "r"), delimiter=",")


#loop through the csv list
for row in csv_file:
    #if current rows 2nd value is equal to input, print that row
    if state == row[1]:
         print (row[2])
         FIP = row[2]
         S_NAME = row[0]

Enter state postal code: (Example for california enter CA)
CA
06


In [ ]:
#Add control polygon of California //  https://datadrivenlab.org/big-data-2/google-earth-engine-tutorial/
# Load US county dataset.
countyData = ee.FeatureCollection('TIGER/2018/Counties').filter(ee.Filter.eq('STATEFP', FIP))

# Get the union of the county geometries in the county.
control = countyData.union(100).geometry().bounds(100)


#Add Earth Engine dataset (NAIP)
naip = ee.ImageCollection("USDA/NAIP/DOQQ").filterDate('2016-01-01','2019-01-01')
naip1= naip.median().clip(control);


In [ ]:
#Add water ocurrence map to see the water bodies
water = ee.Image("JRC/GSW1_2/GlobalSurfaceWater").select('occurrence').clip(control)
water = water


In [ ]:
#Add DEM
DEM = ee.Image("USGS/NED").clip(control)
aspect = ee.Terrain.aspect(DEM)
slope  =ee.Terrain.slope(DEM)


In [ ]:
#Add continents' boundaries to delete polygons on the coastline
continents = ee.Image("users/gena/land_polygons_image")

In [ ]:
# https://developers.google.com/earth-engine/guides/image_visualization 

ndwi = naip1.normalizedDifference(['G','N']).rename(['ndwi'])

# To mask the non-watery parts of the image, where NDWI < 0.3
ndwiMasked = ndwi.updateMask(ndwi.gte(0.5))


## Normalized channels

In [ ]:
SCALE = 10
DEMminMax = DEM.reduceRegion(
  reducer= ee.Reducer.minMax(),
  geometry= DEM.geometry(),
  scale= SCALE,
  maxPixels= 10e12
  )

ndwiminMax = ndwi.reduceRegion(
  reducer= ee.Reducer.minMax(),
  geometry= ndwi.geometry(),
  scale= SCALE,
  maxPixels= 10e12
  )

waterminMax = water.reduceRegion(
  reducer= ee.Reducer.minMax(),
  geometry= water.geometry(),
  scale= SCALE,
  maxPixels= 10e12
  )

AspectminMax = aspect.reduceRegion(
  reducer= ee.Reducer.minMax(),
  geometry= aspect.geometry(),
  scale= SCALE,
  maxPixels= 10e12
  )

SlopeminMax = slope.reduceRegion(
  reducer= ee.Reducer.minMax(),
  geometry= slope.geometry(),
  scale= SCALE,
  maxPixels= 10e12
  )

In [ ]:
norm_DEM = DEM.unitScale(ee.Number(DEMminMax.get('elevation_min')), ee.Number(DEMminMax.get('elevation_max'))).double()
norm_ndwi = ndwi.unitScale(ee.Number(ndwiminMax.get('ndwi_min')), ee.Number(ndwiminMax.get('ndwi_max'))).double()
norm_water = water.unitScale(ee.Number(waterminMax.get('occurrence_min')), ee.Number(waterminMax.get('occurrence_max'))).double() 
norm_aspect = aspect.unitScale(ee.Number(AspectminMax.get('aspect_min')), ee.Number(AspectminMax.get('aspect_max'))).double() 
norm_slope = slope.unitScale(ee.Number(SlopeminMax.get('slope_min')), ee.Number(SlopeminMax.get('slope_max'))).double() 

In [ ]:
#To add the normalized channels as bands in the NAIP image
naip2 = naip1.addBands(norm_ndwi).addBands(norm_water).addBands(norm_aspect).addBands(norm_slope).addBands(norm_DEM)

## Map visualization

In [ ]:
# # Displays the map
# #colors https://en.wikipedia.org/wiki/Web_colors 
# #https://github.com/gee-community/ee-palettes

Map = geemap.Map()
Map.setCenter(-120.08, 36.86)

Map.addLayer(naip1, {}, "NAIP");

#AOI: Area of interest
Map.addLayer(control, {'color': 'yellow', 'opacity': 0.5}, "AOI");

#DEM bands
hs = ee.Terrain.hillshade(DEM)
Map.addLayer(hs, {}, 'Elevation hillshade')
Map.addLayer(aspect, {'min': 0, 'max': 360}, 'Aspect')
Map.addLayer(slope, {}, 'Slope')

#Water occurrence
Map.addLayer(water, { 'min': 0, 'max': 1, 'palette': ['00ffff'] }, 'water')

#NDWI
Map.addLayer(ndwi, {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}, 'NDWI')
# ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']
Map.addLayer(ndwiMasked, {'min': 0.5, 'max': 1, 'palette': ['00FFFF', '0000FF']}, 'NDWI masked')

Map.addLayerControl() 
Map

# Collect dams' dataset from OpenStreetMap API and NID


## Importing dams locations from OpenStreetMap (Overpass API)
OSM API allows the extraction of data in form of features (node, way, relation). In the following algorithm is carried out the extraction of point locations classified as dams in OSM. 

To create the dam's queries with the right tags: https://taginfo.openstreetmap.org/tags

*Note: the extraction of each type of feature has to be done separetely, otherwise the code selects just one type and the other features are not extracted properly.*

In [ ]:
url = 'http://overpass-api.de/api/interpreter'  # Overpass API URL
query = f"""
[out:json];
area["ISO3166-2"="US-{state}"][admin_level=4];
(node["waterway"="dam"](area);
);
out center;
"""

r = requests.get(url, params={'data': query})
data = r.json()['elements']  # read response as JSON and get the data
df = json_normalize(data).rename(columns={"tags.name":"tag_name"}) #creates a dataframe with the data extracted


In [ ]:
url = 'http://overpass-api.de/api/interpreter'  # Overpass API URL
query2 = f"""
[out:json];
area["ISO3166-2"="US-{state}"][admin_level=4];
( way["waterway"="dam"](area);
);
out center;
"""

r2 = requests.get(url, params={'data': query2})
data2 = r2.json()['elements']  # read response as JSON and get the data
df2 = json_normalize(data2).rename(columns={"center.lat":"lat", "center.lon":"lon", "tags.name":"tag_name"})


**Re-run the code below, sometimes presents error on the first run**

In [ ]:
url = 'http://overpass-api.de/api/interpreter'  # Overpass API URL
query3 = f"""
[out:json];
area["ISO3166-2"="US-{state}"][admin_level=4];
( rel["waterway"="dam"](area);
);
out center;
"""

r3 = requests.get(url, params={'data': query3})
data3 = r3.json()['elements']  # read response as JSON and get the data
df3 = json_normalize(data3).rename(columns={"center.lat":"lat", "center.lon":"lon", "tags.name":"tag_name"})

In [ ]:
#Select the columns of the dataframe
nodes = pd.DataFrame(df, columns=['lat','lon', 'type', 'tag_name'])
ways = pd.DataFrame(df2, columns=['lat','lon', 'type', 'tag_name'])
relations = pd.DataFrame(df3, columns=['lat','lon', 'type', 'tag_name'])

#Append the three features and add index
OSM_dams = nodes.append(ways).append(relations)
OSM_dams.reset_index(drop = True, inplace= True)

#Create GeoDataframe of the OSM feature collection
gdf_OSM = geopandas.GeoDataFrame(OSM_dams)

## Importing dams locations from National Inventory of Dams (NID) database

In [ ]:
lines = list()

with open(f'{GDRIVE}/NID.csv', 'r') as readFile:

    reader = csv.reader(readFile)

    for row in reader:

        for field in row:

            if field == state:

                lines.append(row)

with open('mycsv.csv', 'w') as writeFile:

    writer = csv.writer(writeFile)

    writer.writerows(lines)

In [ ]:
data_NID = pd.DataFrame(lines, columns=['RECORDID','LONGITUDE','LATITUDE',
                                        'DAM_TYPE','PURPOSES','DAM_LENGTH','DAM_HEIGHT',
                                        'MAX_STORAGE','SURFACE_AREA','HAZARD','STATE'])
data_NID['LATITUDE']=pd.to_numeric(data_NID['LATITUDE'])
data_NID['LONGITUDE']=pd.to_numeric(data_NID['LONGITUDE'])
data_NID["id"] = data_NID.index

In [ ]:
#Create GeoDataframe of the NID feature collection
gdf_NID=geopandas.GeoDataFrame(data_NID)

In [ ]:
grand = list()

with open(f'{GDRIVE}/GRAND.csv', 'r') as readFile:

    reader = csv.reader(readFile)

    for row in reader:

        for field in row:

            if field == S_NAME:

                grand.append(row)

with open('mycsv.csv', 'w') as writeFile:

    writer = csv.writer(writeFile)

    writer.writerows(grand)

In [ ]:
data_GRAND = pd.DataFrame(grand, columns=['GRAND', 'NAME', 'RIVER', 'STATE', 'YEAR', 
                                          'HEIGHT', 'LEN', 'AREA_SKM', 'CAP_MCM', 
                                          'DIS_AVG_LS', 'MAIN_USE', 'QUALITY', 'LONG', 'LAT'])
data_GRAND['LAT']=pd.to_numeric(data_GRAND['LAT'])
data_GRAND['LONG']=pd.to_numeric(data_GRAND['LONG'])
data_GRAND["id"] = data_GRAND.index

In [ ]:
#Create GeoDataframe of the NID feature collection
gdf_GRAND=geopandas.GeoDataFrame(data_GRAND)

## Create earth engine objects to add to the interactive map

In [ ]:
#To convert the dataframe to shape to then create an earth engine object and add the layer to the map

gdf_OSM.set_geometry(
    geopandas.points_from_xy(gdf_OSM['lon'], gdf_OSM['lat']),
    inplace=True, crs='EPSG:4326')
gdf_OSM.drop(['lat', 'lon'], axis=1, inplace=True)  # optional
gdf_OSM.to_file('dams_OSM.shp')

ee_OSM = geemap.shp_to_ee('dams_OSM.shp')

In [ ]:
gdf_NID.set_geometry(
    geopandas.points_from_xy(gdf_NID['LONGITUDE'], gdf_NID['LATITUDE']),
    inplace=True, crs='EPSG:4326')
gdf_NID.drop(['LATITUDE', 'LONGITUDE'], axis=1, inplace=True)  # optional

gdf_NID[~gdf_NID.geometry.is_valid]
gdf_NID = gdf_NID[gdf_NID.geometry.is_valid == True]
gdf_NID.to_file('dams_NID.shp')

ee_NID = geemap.shp_to_ee('dams_NID.shp')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  


In [ ]:
gdf_GRAND.set_geometry(
    geopandas.points_from_xy(gdf_GRAND['LONG'], gdf_GRAND['LAT']),
    inplace=True, crs='EPSG:4326')
gdf_GRAND.drop(['LAT', 'LONG'], axis=1, inplace=True)  # optional

gdf_GRAND[~gdf_GRAND.geometry.is_valid]
gdf_GRAND = gdf_GRAND[gdf_GRAND.geometry.is_valid == True]
gdf_GRAND.to_file('dams_GRAND.shp')

ee_GRAND = geemap.shp_to_ee('dams_GRAND.shp')

## Create a polygon with centroid in the imported feature collection (OSM/NID)

In [ ]:
buffer = gdf_OSM.buffer(0.01, cap_style=3) #0.02 is the dimension in lat/long and cap_style: 1 (round), 2 (flat), 3 (square)
buffer_NID = gdf_NID.buffer(0.01, cap_style=3)
buffer_GRAND = gdf_GRAND.buffer(0.01, cap_style=3)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#From Geoseries to EE object
buf_OSM = geopandas.GeoDataFrame(geometry=geopandas.GeoSeries(buffer))
buf_OSM.to_file('buffer_OSMdams.shp')
ee_polygon_OSM = geemap.shp_to_ee('buffer_OSMdams.shp')

buf_NID = geopandas.GeoDataFrame(geometry=geopandas.GeoSeries(buffer_NID))
buf_NID.to_file('buffer_NIDdams.shp')
ee_polygon_NID = geemap.shp_to_ee('buffer_NIDdams.shp')

buf_GRAND = geopandas.GeoDataFrame(geometry=geopandas.GeoSeries(buffer_GRAND))
buf_GRAND.to_file('buffer_GRANDdams.shp')
ee_polygon_GRAND = geemap.shp_to_ee('buffer_GRANDdams.shp')


## Map visualization

In [ ]:
# Displays the map
Map = geemap.Map()
Map.setCenter(-120.08, 36.86)
bounds = Map.get_bounds()

Map.addLayer(naip1, {}, "NAIP");

#Add points of OSM and NID feature collections
Map.addLayer(ee_OSM, {'color': 'yellow'}, 'Dams OSM')
Map.addLayer(ee_NID, {'color': 'lime'}, 'Dams NID')
Map.addLayer(ee_GRAND, {'color': 'orange'}, 'Dams GRAND')

#Add unfiltered polygons of OSM and NID dams
Map.addLayer(ee_polygon_OSM, {'color': 'yellow', 'opacity': 0.5}, 'Polygon_dams_OSM')
Map.addLayer(ee_polygon_NID, {'color': 'lime', 'opacity': 0.5}, 'Polygon_dams_NID')
Map.addLayer(ee_polygon_GRAND, {'color': 'orange', 'opacity': 0.5}, 'Polygon_dams_GRAND')

Map.addLayerControl() 
Map

# Refine the dam dataset 

Remove data that can be noisy 

## Filtering functions

In [ ]:
def compute_water_area(f):
    water2 = water
    Waterarea = ee.Image.pixelArea().mask(water2).reduceRegion(
      reducer= ee.Reducer.sum(), 
      geometry= f.geometry(), 
      scale= SCALE)
  
    return f.set(Waterarea)

In [ ]:
#Function to verify point-in-polygon (pip)
#Adapted from: https://medium.com/analytics-vidhya/point-in-polygon-analysis-using-python-geopandas-27ea67888bff 

def get_polygon_in_polygon (gdf, polygon):
    id_list = list(polygon.FID)
    #create empty dataframe
    df = pd.DataFrame().reindex_like(gdf).dropna()
    for i in id_list:
        #get geometry for specific region
        pol = (polygon.loc[polygon.FID==i])
        pol.reset_index(drop = True, inplace = True)
        #identify those records from gdf that are intersecting with the region polygon
        pip_mask = gdf.intersects(pol.loc[0, 'geometry'])
        #filter gdf to keep only the intersecting records
        pip_data = gdf.loc[pip_mask].copy()
        #create a new column and assign the FID as the value
        pip_data['FID']= i
        #append region data to empty dataframe
        df = df.append(pip_data)
        
    #checking there are no more than one region assigned to a point  
    print('Original dataframe count=',len(gdf),'\nNew dataframe count=', len(df))
    if df.loc[df.id.duplicated() == True].shape[0] > 0:
        print("There are id's with more than one region")

    #checking all points have a region
    elif gdf.loc[~gdf.id.isin(df.id)].shape[0] > 0:
        print("There are id's without an assigned region")
    else:
        print("No discrepancies in results!")

    df.reset_index(inplace=True)

    return df

In [ ]:
def in_land(f):
  isInland = continents.mask().reduceRegion(
      reducer= ee.Reducer.allNonZero(),
      geometry= f.geometry(),
      scale= 30).values().get(0)

  return f.set({'isInland': isInland})


## Double validate location and remove remaining patches
Since there are many overlapping patches from the two datasets imported (OSM and NID) in this section are filtered the overlapping patches and are removed the remaining patches. In this way, the location is double confirmed and it is reduce the risk of having non-dams included in the final dataset.

In [ ]:
buf_OSM['FID'] = buf_OSM.index
buf_NID['FID'] = buf_NID.index+len(buf_OSM)
buf_GRAND['FID'] = buf_GRAND.index+(len(buf_OSM)+len(buf_NID))
buf_OSM['id'] = buf_OSM.index
buf_NID['id'] = buf_NID.index+len(buf_OSM)
buf_GRAND['id'] = buf_GRAND.index+(len(buf_OSM)+len(buf_NID))


In [ ]:
overlay_polygons = get_polygon_in_polygon(buf_NID, buf_OSM)

Original dataframe count= 1573 
New dataframe count= 2461
There are id's with more than one region


In [ ]:
overlay_polygons.drop_duplicates('id', inplace= True)

In [ ]:
# #From geodataframe to GEE features
gdf_all = geopandas.GeoDataFrame(overlay_polygons)
gdf_all.to_file('all_NID.shp')
ee_all = geemap.shp_to_ee('all_NID.shp')

In [ ]:
# # Displays the map
Map = geemap.Map()
Map.setCenter(-120.08, 36.86)

Map.addLayer(naip1, {}, "NAIP");

#Add unfiltered polygons of OSM and NID dams
Map.addLayer(ee_polygon_OSM, {'color': 'yellow', 'opacity': 0.5}, 'Polygon_dams_OSM')
Map.addLayer(ee_polygon_NID, {'color': 'lime', 'opacity': 0.5}, 'Polygon_dams_NID')
Map.addLayer(ee_polygon_GRAND, {'color': 'orange', 'opacity': 0.5}, 'Polygon_dams_GRAND')

Map.addLayer(ee_all, {'color': 'blue', 'opacity': 0.5}, 'Polygon_dams')

Map.addLayerControl() 
Map

## Filter dam polygons by water area

In [ ]:
### Now we filter the squares for which area does not reach a certain threshold
ee_dams_area = ee_all.map(compute_water_area)

AreaThreshold = 80000
ee_dams_water_filter = ee_dams_area.filter(ee.Filter.gt('area', AreaThreshold))

print('All polygons OSM #{}'.format(ee_dams_area.size().getInfo()))
print('Filtered polygons OSM by area #{}'.format(ee_dams_water_filter.size().getInfo()))


All polygons OSM #1298
Filtered polygons OSM by area #768


In [ ]:
# Displays the map
Map = geemap.Map()
Map.setCenter(-120.08, 36.86)
bounds = Map.get_bounds()

Map.addLayer(naip1, {}, "NAIP");

Map.addLayer(water, { 'min': 0, 'max': 1, 'palette': ['aqua'], 'opacity': 0.4 }, 'water')

#Add unfiltered polygons of OSM and NID dams
Map.addLayer(ee_all, {'color': 'red', 'opacity': 0.3}, 'Polygon_dams')
Map.addLayer(ee_polygon_GRAND, {'color': 'orange', 'opacity': 0.5}, 'Polygon_dams_GRAND')

#Filtered polygons
Map.addLayer(ee_dams_water_filter, {'color': 'blue', 'opacity': 0.6}, 'Waterfilter_dams')

# Map.addLayer(ee_dams.filter(ee.Filter.eq('isInland', 1)), {'color' : 'yellow'}, 'coastline')


Map.addLayerControl() 
Map

## Delete polygons on the coastline

In [ ]:
ee_dams = ee_dams_water_filter.map(in_land).filter(ee.Filter.eq('isInland', 1))

print(ee_dams.aggregate_array('isInland').getInfo())


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
dams_grand_filter = geopandas.GeoDataFrame.from_features(ee_dams.getInfo())
overlay_polygons_2 = get_polygon_in_polygon(dams_grand_filter, buf_GRAND)
overlay_polygons_2.drop_duplicates('FID', inplace= True)

Original dataframe count= 759 
New dataframe count= 221
There are id's without an assigned region


In [ ]:
ind_ = []
for n in range(len(buf_GRAND)):
  for i in overlay_polygons_2['FID']:
    if int(i) == buf_GRAND['FID'][n]:
      ind_.append(n)

pol_GRAND = buf_GRAND.drop(ind_)

print(pol_GRAND.index)

Int64Index([52, 80, 97, 109, 112], dtype='int64')


In [ ]:
final_dams = dams_grand_filter.append(pol_GRAND)
final_dams.to_file('all_dams.shp')
ee_final_dams = geemap.shp_to_ee('all_dams.shp')

/usr/local/lib/python3.7/dist-packages/geemap/common.py:6873: UserWarning: The projection file /content/all_dams.prj could not be found. Assuming the dataset is in a geographic coordinate system (GCS).
  f"The projection file {in_prj} could not be found. Assuming the dataset is in a geographic coordinate system (GCS)."


In [ ]:
# total_dams = geopandas.GeoDataFrame.from_features(ee_dams.getInfo())
# dams = total_dams.centroid

In [ ]:
# buf_dams = dams.buffer(0.02, cap_style= 3)
# buf_dams = geopandas.GeoDataFrame(geometry=geopandas.GeoSeries(buf_dams))
# buf_dams.to_file('all_dams.shp')
# ee_final_dams = geemap.shp_to_ee('all_dams.shp')

In [ ]:
# Displays the map
Map = geemap.Map()
Map.setCenter(-120.08, 36.86)
bounds = Map.get_bounds()

Map.addLayer(naip1, {}, "NAIP");

Map.addLayer(water, { 'min': 0, 'max': 1, 'palette': ['aqua'], 'opacity': 0.4 }, 'water')

#Final patches
Map.addLayer(ee_final_dams, {'color': 'blue', 'opacity': 0.8}, 'Final_dams')

Map.addLayerControl() 
Map

# Collect negative images (No dam) dataset 
To produce the negative images (non-dam) the dataset is build using bridge locations (since are linear structures that can be mistaken for dams), lakes and random points

In [ ]:
random_points = ee.FeatureCollection.randomPoints(
    region= control,
    points= 200,
    seed= 1234
)

random_points = geopandas.GeoDataFrame.from_features(random_points.getInfo())
random_points["id"] = random_points.index

In [ ]:
url = 'http://overpass-api.de/api/interpreter'  # Overpass API URL
query = f"""
[out:json];
area["ISO3166-2"="US-{state}"][admin_level=4];
(way["bridge"="yes"](area);
);
out center;
"""

rb = requests.get(url, params={'data': query})
data_b = rb.json()['elements']  # read response as JSON and get the data
df_b = json_normalize(data_b).rename(columns={"center.lat":"lat", "center.lon":"lon"})


In [ ]:
url = 'http://overpass-api.de/api/interpreter'  # Overpass API URL
query = f"""
[out:json];
area["ISO3166-2"="US-{state}"][admin_level=4];
(way["water"="lake"](area);
 way["natural"="water"](area);
);
out center;
"""

rl = requests.get(url, params={'data': query})
data_l = rl.json()['elements']  # read response as JSON and get the data
df_l = json_normalize(data_l).rename(columns={"center.lat":"lat", "center.lon":"lon"})

In [ ]:
bridges = pd.DataFrame(df_b, columns=['lat','lon', 'id'])
bridges.to_csv(r'osm_bridge.csv', encoding='utf-8', index=False)

lakes = pd.DataFrame(df_l, columns=['lat','lon', 'id'])
lakes.to_csv(r'osm_lake.csv', encoding='utf-8', index=False)

In [ ]:
gdf_bridge = geopandas.GeoDataFrame(bridges)

#To convert the dataframe to shape to then create an earth engine object and add the layer to the map
gdf_bridge.set_geometry(
    geopandas.points_from_xy(gdf_bridge['lon'], gdf_bridge['lat']),
    inplace=True, crs='EPSG:4326')
gdf_bridge.drop(['lat', 'lon'], axis=1, inplace=True)  # optional
gdf_bridge.to_file('bridges_osm.shp')

In [ ]:
gdf_lake = geopandas.GeoDataFrame(lakes)

#To convert the dataframe to shape to then create an earth engine object and add the layer to the map
gdf_lake.set_geometry(
    geopandas.points_from_xy(gdf_lake['lon'], gdf_lake['lat']),
    inplace=True, crs='EPSG:4326')
gdf_lake.drop(['lat', 'lon'], axis=1, inplace=True)  # optional
gdf_lake.to_file('lakes_osm.shp')

In [ ]:
random_buffer = random_points.buffer(0.01, cap_style=3)
random_buffer = geopandas.GeoDataFrame(geometry=geopandas.GeoSeries(random_buffer))
random_buffer['id'] = random_buffer.index


In [ ]:
bridge_buffer = gdf_bridge.buffer(0.01, cap_style=3)
bridge_buffer = geopandas.GeoDataFrame(geometry=geopandas.GeoSeries(bridge_buffer))
bridge_buffer['id'] = bridge_buffer.index

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


In [ ]:
lakes_buffer = gdf_lake.buffer(0.01, cap_style=3)
lakes_buffer = geopandas.GeoDataFrame(geometry=geopandas.GeoSeries(lakes_buffer))
lakes_buffer['id'] = lakes_buffer.index

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.


# Refine no dam dataset

To make the no-dam images more representative for the model to learn

In [ ]:
all_dams = buf_OSM.append(buf_NID).append(buf_GRAND)

In [ ]:
lake_filter = get_polygon_in_polygon(lakes_buffer, all_dams)

Original dataframe count= 2443 
New dataframe count= 947
There are id's with more than one region


In [ ]:
lake_filter.sort_values(by=['id'])

,index,geometry,id,FID
340,0,"POLYGON ((-116.90663 32.63818, -116.90663 32.6...",0.0,1373.0
538,0,"POLYGON ((-116.90663 32.63818, -116.90663 32.6...",0.0,2363.0
31,2,"POLYGON ((-122.20837 37.83255, -122.20837 37.8...",2.0,190.0
146,2,"POLYGON ((-122.20837 37.83255, -122.20837 37.8...",2.0,492.0
145,2,"POLYGON ((-122.20837 37.83255, -122.20837 37.8...",2.0,491.0
...,...,...,...,...
542,2367,"POLYGON ((-122.44447 37.77910, -122.44447 37.7...",2367.0,2386.0
85,2407,"POLYGON ((-118.54306 37.16608, -118.54306 37.1...",2407.0,310.0
645,2407,"POLYGON ((-118.54306 37.16608, -118.54306 37.1...",2407.0,2717.0
527,2423,"POLYGON ((-119.36836 37.58950, -119.36836 37.5...",2423.0,2291.0


In [ ]:
if lake_filter['id'].count() == 0:
  lakes_gdf = lakes_buffer
else:
  indx = []
  for i in lake_filter['index']:
    indx.append(i)
  indx.sort()
  lakes_gdf = lakes_buffer.drop(indx, axis=0)


In [ ]:
bridge_filter = get_polygon_in_polygon(bridge_buffer, all_dams)

Original dataframe count= 43684 
New dataframe count= 19836
There are id's with more than one region


In [ ]:
bridge_filter.drop_duplicates('id', inplace=True)
if bridge_filter['id'].count() == 0:
  bridge_gdf = bridge_buffer
else:
  indx_b = []
  for i in bridge_filter['index']:
    indx_b.append(i)
  indx_b.sort()
  bridge_gdf = bridge_buffer.drop(indx_b, axis=0)

In [ ]:
random_filter = get_polygon_in_polygon(random_buffer, all_dams)

Original dataframe count= 200 
New dataframe count= 29
There are id's with more than one region


In [ ]:
if random_filter['id'].count() == 0:
  random_gdf = random_buffer
else:
  indx_r = []
  for i in random_filter['index']:
    indx_r.append(i)
  random_gdf = random_buffer.drop(indx_r, axis=0)

In [ ]:
lakes_gdf['FID'] = lakes_gdf.index
bridge_gdf['FID'] = bridge_gdf.index+len(lakes_gdf)
random_gdf['FID'] = random_gdf.index+(len(lakes_gdf)+len(bridge_gdf))

In [ ]:
lakes_gdf = lakes_gdf.sample(800, random_state=1234)
bridge_gdf = bridge_gdf.sample(2000, random_state=1234)

In [ ]:
lake_n_brg = get_polygon_in_polygon(lakes_gdf, bridge_gdf)

Original dataframe count= 800 
New dataframe count= 77
There are id's with more than one region


In [ ]:
lake_n_brg.drop_duplicates('id', inplace=True)

In [ ]:
id_ = []
for i in lake_n_brg['id']:
  id_.append(int(i))


In [ ]:
lakes_gdf.drop(id_, inplace=True)

In [ ]:
random_n_brg = get_polygon_in_polygon(bridge_gdf, random_gdf)

Original dataframe count= 2000 
New dataframe count= 4
There are id's without an assigned region


In [ ]:
idr_ = []
for i in random_n_brg['id']:
  idr_.append(int(i))

bridge_gdf.drop(idr_, inplace=True)

In [ ]:
lakes_gdf.to_file('lake_filter.shp')
ee_lakes = geemap.shp_to_ee('lake_filter.shp')

bridge_gdf.to_file('bridge_filter.shp')
ee_bridge = geemap.shp_to_ee('bridge_filter.shp')

random_gdf.to_file('random_filter.shp')
ee_random = geemap.shp_to_ee('random_filter.shp')

/usr/local/lib/python3.7/dist-packages/geemap/common.py:6873: UserWarning: The projection file /content/random_filter.prj could not be found. Assuming the dataset is in a geographic coordinate system (GCS).
  f"The projection file {in_prj} could not be found. Assuming the dataset is in a geographic coordinate system (GCS)."


## Filter no-dam polygons by water area

In [ ]:
### Now we filter the squares for which area does not reach a certain threshold
ee_bridge_area = ee_bridge.map(compute_water_area)
ee_lakes_area = ee_lakes.map(compute_water_area)

ee_bridge_w = ee_bridge_area.filter(ee.Filter.gt('area', AreaThreshold))
ee_lakes_w = ee_lakes_area.filter(ee.Filter.gt('area', AreaThreshold))

print('All polygons OSM #{}'.format(ee_bridge.size().getInfo()))
print('Filtered polygons OSM by area #{}'.format(ee_bridge_w.size().getInfo()))

print('All polygons OSM #{}'.format(ee_lakes.size().getInfo()))
print('Filtered polygons OSM by area #{}'.format(ee_lakes_w.size().getInfo()))

All polygons OSM #1996
Filtered polygons OSM by area #392
All polygons OSM #759
Filtered polygons OSM by area #343


# VISUALIZATION

In [ ]:
# # Displays the map
Map = geemap.Map()
Map.setCenter(-120.08, 36.86)
bounds = Map.get_bounds()

Map.addLayer(naip1, {}, "NAIP");

Map.addLayer(water, { 'min': 0, 'max': 1, 'palette': ['aqua'], 'opacity': 0.4 }, 'water')

#Final patches
Map.addLayer(ee_final_dams, {'color': 'blue', 'opacity': 0.8}, 'Final_dams')


Map.addLayer(ee_bridge_w, {'color': 'pink', 'opacity': 0.6}, 'bridges')

Map.addLayer(ee_lakes_w, {'color': 'PaleGreen', 'opacity': 0.6}, 'lakes area')

Map.addLayer(ee_random, {'color': 'silver', 'opacity': 0.6}, 'random')

Map.addLayerControl() 
Map

# Export patches as TFRecords to Drive

## Export dam images

In [ ]:
# rect_list = ee_final_dams.toList(10000)

# count = rect_list.size().getInfo()

# SIZE = 440
# for i in range(count):
#   rect = ee.Feature(rect_list.get(i)).geometry()

#   task = ee.batch.Export.image.toDrive(  
#     image= naip2,
#     description= 'PatchesExported-%d'%(i),
#     fileNamePrefix= 'Dam-%d'%(i),
#     folder= f'Dam_{state}',
#     region= rect,
#     scale= SCALE,
#     maxPixels=10e12,
#     fileFormat= 'TFRecord',
#     formatOptions = {
#       'patchDimensions': [SIZE, SIZE],
#       'compressed': True}
#   ).start()

#   print('Export Image %d was submitted, please wait ...'%(i))




## Export non-dam images

In [ ]:
# random_list = ee_random.toList(100)
# bridge_list = ee_bridge_w.toList(400)
# lake_list = ee_lakes_w.toList(200)

# for i in range(100):

#   random = ee.Feature(random_list.get(i)).geometry()

#   # // EXPORT PATCHES

#   task = ee.batch.Export.image.toDrive(  
#     image= naip2,
#     description= 'RandomPatches-%d'%(i),
#     fileNamePrefix= 'random-%d'%(i),
#     scale= SCALE,
#     folder= f'No_dam_{state}_tfr',
#     fileFormat= 'TFRecord',
#     region= random,
#     formatOptions = {
#       'patchDimensions': [SIZE, SIZE],
#       'compressed': True}
#   ).start()

#   print('Export Random Image %d was submitted, please wait ...'%(i))



# for i in range(400):

#   bridges = ee.Feature(bridge_list.get(i)).geometry()

#   task = ee.batch.Export.image.toDrive(  
#     image= naip2,
#     description= 'RandomPatches-%d'%(i),
#     fileNamePrefix= 'bridges-%d'%(i),
#     scale= SCALE,
#     folder= f'No_dam_{state}_tfr',
#     fileFormat= 'TFRecord',
#     region= bridges,
#     formatOptions = {
#       'patchDimensions': [SIZE, SIZE],
#       'compressed': True}
#   ).start()

#   print('Export Bridge Image %d was submitted, please wait ...'%(i))



# for i in range(200):

#   lakes = ee.Feature(lake_list.get(i)).geometry()

#   task = ee.batch.Export.image.toDrive(  
#     image= naip2,
#     description= 'RandomPatches-%d'%(i),
#     fileNamePrefix= 'lakes-%d'%(i),
#     scale= SCALE,
#     folder= f'No_dam_{state}_tfr',
#     fileFormat= 'TFRecord',
#     region= lakes,
#     formatOptions = {
#       'patchDimensions': [SIZE, SIZE],
#       'compressed': True}
#   ).start()

#   print('Export Lake Image %d was submitted, please wait ...'%(i))



# Export patches as GeoTIFF to Drive

## Export dam images

In [ ]:
# rect_list = ee_final_dams.toList(10000)

# count = rect_list.size().getInfo()

# SIZE = 220
# for i in range(count):
#   rect = ee.Feature(rect_list.get(i)).geometry()

#   task = ee.batch.Export.image.toDrive(  
#     image= naip2,
#     description= 'DamsExported-%d'%(i),
#     fileNamePrefix= 'Dam-%d'%(i),
#     folder= f'Dam_{state}_tif',
#     region= rect,
#     scale= SCALE,
#     maxPixels=10e12,
#     fileFormat= 'GeoTIFF',

#   ).start()

#   print('Export Image %d was submitted, please wait ...'%(i))



## Export non-dam images

In [ ]:
# r = 100
# b = 350
# l = 200

# random_list = ee_random.toList(r)
# bridge_list = ee_bridge_w.toList(b)
# lake_list = ee_lakes_w.toList(l)

# for i in range(r):

#   random = ee.Feature(random_list.get(i)).geometry()

#   # // EXPORT PATCHES

#   task = ee.batch.Export.image.toDrive(  
#     image= naip2,
#     description= 'RandomPatches-%d'%(i),
#     fileNamePrefix= 'random-%d'%(i),
#     scale= SCALE,
#     folder= f'No_dam_{state}_tif',
#     fileFormat= 'GeoTIFF',
#     region= random,

#   ).start()

#   print('Export Random Image %d was submitted, please wait ...'%(i))



# for i in range(b):

#   bridges = ee.Feature(bridge_list.get(i)).geometry()

#   task = ee.batch.Export.image.toDrive(  
#     image= naip2,
#     description= 'RandomPatches-%d'%(i),
#     fileNamePrefix= 'bridges-%d'%(i),
#     scale= SCALE,
#     folder= f'No_dam_{state}_tif',
#     fileFormat= 'GeoTIFF',
#     region= bridges,

#   ).start()

#   print('Export Bridge Image %d was submitted, please wait ...'%(i))



# for i in range(l):

#   lakes = ee.Feature(lake_list.get(i)).geometry()

#   task = ee.batch.Export.image.toDrive(  
#     image= naip2,
#     description= 'RandomPatches-%d'%(i),
#     fileNamePrefix= 'lakes-%d'%(i),
#     scale= SCALE,
#     folder= f'No_dam_{state}_tif',
#     fileFormat= 'GeoTIFF',
#     region= lakes,

#   ).start()

#   print('Export Lake Image %d was submitted, please wait ...'%(i))

